# Two-step-task

#### The following is a summary of the methods developed by Wang et al. (2016 and 2018) for the two-step-task

In 2016, [Wang et al.](https://arxiv.org/pdf/1611.05763v1.pdf) presented an architecture for "Deep Meta Reinforcement Learning", using a recurrent neural network of type Long-Short-Term Memory (LSTM) and Advantage Actor Critic (A2C). Their goal was to establish whether meta-RL could: 
- Deal with the exploration/exploitation trade-off,
- Understand the abstract structure of a task.

Deep Meta Reinforcement Learning is about training an agent on a distribution of tasks that share a similar structure, for instance interrelated Markov Decision Processes (MDPs).
 

### The Two-Step Task

The task we reproduced is a variation of the [two-step task](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3077926/). In the first-stage state $S_1$ the agent can take one of two different actions ($A_1$, $A_2$), each one leading to the two second-stage states ($S_2$ and $S_3$) with correlated distributions (see the figure below). Once in a second stage-state, the agent gets a reward of $1$ with probability $(p, 1-p)$ in states $(S_2,S_3)$ respectively, with $p  \in \{0.1, 0.9\}$. This reward probability is randomly reset at each episode.

![Two-Step Task](ressources/two_step_task.png)

The actual transition probabilities between first and second stage depend on the paper (in [Wang et al.](https://arxiv.org/pdf/1611.05763v1.pdf) they use [0.75, 0.25] but in [Wang et al., 2018](https://www.biorxiv.org/content/biorxiv/early/2018/04/13/295964.full.pdf) they use [0.8,0.2]).

The two-step task was used because it brings out the behaviour of the RL algorithm; it dissociates model-based algorithms from model-free ones. 

#### Model-based vs model-free

When an agent performs a model-based algorithm, it learns an internal model of the environment he's in (transition and reward functions) and relies on it when planning his next move. On the other hand, model-free algorithms don't store any information of the model but map values to actions in states through local updates, relying on the prediction error (i.e. the difference between the predicted value for that state and the actual outcome). We say that model-based is goal-oriented and model-free is more of an habitual behaviour.

The two-step task makes it easier to distinguish between these two methods because an agent following a model-based strategy takes into account the transition probability of the actions whereas an agent following a model-free algorithm focuses on the reward only. For instance, if a model-free RL agent gets a reward at his last trial performing $A_1$, he will repeat it at his current trial regardless of whether the transition he went through was common ($75\%$) or rare ($25\%$). On the other hand, a model-based RL agent will repeat his last action only if the transition was common. 

A common method used to evaluate this behaviour is the stay probability.


#### The stay probability

The stay probability is the probability of repeating the same action performed in the last trial, given that the last transition was common (with the highest probability of reaching the state that resulted in a positive outcome) or rare, and rewarded or not. An agent following a model-free algorithm repeats his last action as long as it brings him a reward, regardless of the type of the transition he goes through (common/rare), so the plot gives two bars equally high when the last trial is rewarded. On the other hand, a model-based agent repeats more often actions that entails rewards when the transition is common, so the probability of repeating the common transition bar is higher than the rare one.


![Model-based Vs Model-free plot](ressources/stayprob_plots.png)
 <center>Canonical pattern of behavior for model-free (left) and model-based (right) learning ([source](https://www.biorxiv.org/content/biorxiv/early/2018/04/13/295964.full.pdf)) </center>


#### Network

The inputs and outputs for the LSTM are for each time-step:

![Architecture of the network](ressources/architecture.png)

where the inputs are:
- $t$ : current timestep
- $x_t$ : observation at timestep t.
- $r_{t-1}$ : last reward.
- $a_{t-1}$ : last action.

and returns two outputs: 
- $V$ : value of the current state,
- $\pi$ : the softmax policy distribution.

#### Training process

We used the Asynchronous Advantage Actor-Critic (A3C) implemented by Juliani, and launched it with only one worker. A2C being the synchronous version of A3C, using a single thread (worker) gives the same result for both algorithms. 

Wang et al. published two papers describing their method. The main difference is that their [first paper](https://arxiv.org/pdf/1611.05763v1.pdf) ran 10 trials per episode, with the two step task being reset at each episode (the reward probabilities are fixed during an episode), while the [second](https://www.biorxiv.org/content/biorxiv/early/2018/04/13/295964.full.pdf) ran episodes of 200 trials, with the reward probabilities switching with a probability of $2.5\%$. This second paper contains more information about the parameters used in training, like the discount factor.

For their [first paper](https://arxiv.org/pdf/1611.05763v1.pdf), the number of episodes for training and testing are not mentioned. For the [second paper](https://www.biorxiv.org/content/biorxiv/early/2018/04/13/295964.full.pdf) the training is 10,000 episodes and testing is 300 episodes.